In this Notebook, we will do basic minimum with spark streaming.<br/>We will save tweets to mongodb.
<br/> Spark required java 8

### Before starting this code run 
Change the hashtags in variable WORDS<br/>
run : python tweepy_connection.py

In [1]:
# Give path to the spark directory.
import findspark
findspark.init('/home/ashwani/spark/spark-2.4.3-bin-hadoop2.7')

In [2]:
#Import Modules
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext

import json
from pymongo import MongoClient

In [3]:
#Mongo db Connection
MONGO_HOST= 'mongodb://172.18.0.2:27017/'
client = MongoClient(MONGO_HOST)
db = client.twitterdb
coll = db.general

In [4]:
# Create a local StreamingContext and batch interval of 10 seconds
conf = SparkConf().setAppName("Spark Streaming")
sc = SparkContext(conf = conf)
ssc = StreamingContext(sc, 10)

In [5]:
print(sc.uiWebUrl)

http://192.168.1.103:4040


In [6]:
# Create a DStream that will connect to hostname:port
sparkStream = ssc.socketTextStream("localhost", 9010)

## Dstream - sparkStream ##
Dstream is a collection of RDD saved on cache as a batch process.<br/>Each RDD will be a list of twitter data. the list can be of any length.<br/>The object type of element in list is 'str'. We use json.loads to convert it to dict object.

In [7]:
def f(rdd):
    for t in rdd.collect():
        coll.insert_one(json.loads(t))

In [8]:
sparkStream.foreachRDD(lambda rdd: f(rdd))

ssc.start()

In [9]:
ssc.stop()